In [ ]:
## imports
import os 
import sys
import json
from datetime import datetime  
import numpy as np
%matplotlib nbagg
import matplotlib.pyplot as plt

sys.path.insert(0, '../')
import OCT_lib

In [ ]:
### Files management
## You may provide the `path` to the folder with all the binary files
basepath = r"..//02-ExampleData//PDMS_phantom" 
files, basepath = OCT_lib.get_files(initdir = basepath)
folder_name = os.path.basename(os.path.normpath(basepath))

In [ ]:
### Analysis parameters

# default parameters are stored in OCT_lib.py
default_analysis_settings = True
# if enabled, prompts the selection of JSON settings file
read_parameters_from_file = False

if default_analysis_settings:
    p = OCT_lib.default_settings
    print(f"The parameters used for this analysis are the default ones")
elif read_parameters_from_file:
    param_textfile = OCT_lib.get_file(initdir = basepath)
    print(f"The parameters used for this analysis are taken from: \n{param_textfile}") 
    with open(param_textfile, 'r') as file:
        p = json.loads(file.read()) 
else:
    print(f"The parameters used for this analysis are specified in the Jupyter Notebook") 
    ## dictionary will all the parameters for the analysis
    p = dict(
        ## if crop['switch'] is True, the image gets cropped at the specified indices 
        crop = dict(
            # turn the crop switch ON or OFF
            switch = True,
            ## cropping indices
            left = None,  # N of columns removed from the leftmost side
            right = None, # N of columns removed from the rightmost side
            top = 300, # N of columns removed from the top
            bottom = -200, # N of columns removed from the bottom
        ),
        ## if True, the image is resized with the specified multiplicative factors 
        resize = dict(
            switch = False,
            width_factor = 1,
            height_factor = 1,
        ),
        ## `filtering` contains all the optimized parameters for each filter mode
        filtering = dict(
                switch = True, # Turns filtering ON or OFF 
                mode = 'bilateral', # filtering mode: set to `NLM` or `bilateral`
                bilateral = dict(
                    d = 15,
                    sigma_color = 90,
                    sigma_space = 80,
                ),
                NLM = dict(
                    sigma = 6, #parameter for NLM denoising
                ),
            ),
        ## Parameters of segmentation
        segmentation = dict(
            switch = True, 
            mode = "otsu", # OPTIONS  "chan_vese", "otsu" or "max"
            ignore_top_px = 50,
            N_iter = 20, # used in ACWE segm. (see docs of `morphological snakes`)
        ),
        ## Parameters of Axsun OCT
        OCT = dict(
            axial_res = 5.75, # Vertical Resolution of OCT, in micron per pixel
            aperture_size = 4, # size of the suction aperture
            bseypassmode = 8, # this translates to how many bytes per px  (defaul 1b/px): See Axsun manual
        ),           
        save_png = True, # saves B-scans in .png format in folder \processed
        make_res = True, # stores all processing results in a dictionary
        save_zip = True, # saves that dictionary in a pickled .bz2 
        save_settings = False, # saves analysis settings in a JSON file
    )

print(OCT_lib.print_about_settings(p))
      
if p['save_zip']:         
     assert p['make_res'], \
        "Zip file contents are based on the stored results, please set `True` the p['make_res'] switch"  

In [ ]:
# initialization of the dictionary that will store the processed results
if p['make_res']:
    res = dict.fromkeys(files)

# creates folder `processed` in which to save B-scans, parameters file and/or zip file
if (p['save_png'] or p['save_zip']):
    if not os.path.exists(os.path.join(basepath, 'processed')):
        os.makedirs(os.path.join(basepath, 'processed'))

# ==== Cycling over the files ====
for i, file in enumerate(files[:]):
    print("============================\n============================")
    print(f'File "{file}" - {i+1}/{len(files)}')
    path2file = os.path.join(basepath, file)
    print(path2file)
    # load file
    debug_here = True # to show more information during execution
    b = OCT_lib.bscan(path=path2file, debug=debug_here)

    # crop
    if p['crop']['switch']:
        b.crop(top=p['crop']['top'],
               bottom=p['crop']['bottom'],
               left=p['crop']['left'],
               right=p['crop']['right'])

    ## halves the size - used for 3D volumetric visualization:
    if p['resize']['switch']:
        b.resize_raw(w=b.width * p['resize']['width_factor'],
                     h=b.ascanlength * p['resize']['height_factor'])

    ## filters the image if required
    if p['filtering']['mode']:
        # The next line is a bit complicated. Apply_filter takes in some keyword arguments
        # that are packed into the settings dictionary p['filtering'][filtermode]
        b.apply_filter(mode=p['filtering']['mode'],
                       **p['filtering'][p['filtering']['mode']])

    ## Surface identification, returns a 1D array with the height of the surface
    if p['segmentation']['switch']:
        profile = b.get_profile(mode=p['segmentation']['mode'],
                                exclude_top_px=p['segmentation']['ignore_top_px'],
                                smoothed = True
                               )

    ## To obtain the 2D mask (called levelset), use:
#         levelset = b.get_level_set(mode='otsu')

    if p['make_res']:
        res[file] = {}
        
        res[file]['image'] = b.raw
        # relaxed or suction?
        if 'suc' in file:
            res[file]['pressure'] = 'suction'
        elif 'rel' in file:
            res[file]['pressure'] = 'relaxed'

        if p['segmentation']['switch']:
            if 'profile' in locals():
                # 1D array of skin height along the B-scan
                res[file]['profile'] = profile
            if 'levelset' in locals():
                # B&W image returned by segmentation
                res[file]['levelset'] = levelset

    # saving png image
    if p['save_png']:
        png_name = file
        profile = profile if p['segmentation']['switch'] else None
        b.save_png(
            name = png_name,
            profile=profile,
        )
    print("\n")
    
# === END OF FILEs CYCLING ===

# Reads the time at which the analysis has been executed
analysis_time = datetime.now().strftime("%Y%m%d_%H-%M")

# also save analysis parameters in a .txt file (for further use)
if p['save_settings']:
    settings_filename = f"{analysis_time}_parameters.json"
    settings_pathfile = os.path.join(
        basepath, 'processed', settings_filename)
    print(
        f"The parameters used for this analysis have been written at: \n{settings_pathfile}")
    with open(settings_pathfile, 'w') as file: 
        # use `json.loads` to do the reverse
        file.write(json.dumps(p, indent=3))

# save all processed data in a zip file
if p['save_zip']:
    experiment = {}
    experiment['results'] = res
    experiment['params'] = p
    experiment['name'] = folder_name 
    import bz2  # to zip the pickle file:
    import pickle
#     pickle_name = (os.path.join(basepath,'processed'))+'\\'+folder_name'+'_processed.bz2'
#     pickle_name = f"{os.path.join(basepath,'processed')}\\{folder_name}_{analysis_time}_processed.bz2"
    pickle_name = os.path.join(basepath,'processed',f"{folder_name}_{analysis_time}_processed.bz2")   
    print(f"Saving file at path: {pickle_name}")
    sfile = bz2.BZ2File(pickle_name, 'w')
    pickle.dump(experiment, sfile)
    sfile.close()